In [3]:
import pymysql.cursors
import sys
import pandas as pd
import pymysql

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='db_gdelt_fintech',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}


In [ ]:
with connection.cursor() as cursor:
    sql = "SELECT GLOBALEVENTID, sourceurl FROM gojek"
    cursor.execute(sql)
    #print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r_split = r["sourceurl"].split('//')
            #print(r_split[0])
            r2_split = r_split[1].split('/')
            #print(r2_split[0])
            base_url = r_split[0]+"//"+r2_split[0]
            print(base_url)
            with connection.cursor() as cursor:
                sql = "UPDATE gojek SET base_url=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (base_url.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

In [4]:
def run(sql):
    df = pd.read_sql_query(sql,connection)
    return df

In [5]:
test = run("select SOURCEURL from tm_fintech_all_clean")
test

,SOURCEURL
0,http://spectrum.ieee.org/tech-talk/computing/n...
1,http://www.independent.ie/world-news/police-fo...
2,http://www.coindesk.com/indonesia-central-bank...
3,http://www.independent.ie/business/world/the-r...
4,http://www.thejakartaglobe.com/business/bitcoi...
5,http://au.news.yahoo.com/thewest/business/tech...
6,http://www.straitstimes.com/breaking-news/mone...
7,http://newsbtc.com/2014/02/06/bank-indonesia-i...
8,http://thenextweb.com/asia/2014/02/06/bank-ind...
9,http://www.coindesk.com/indonesia-central-bank...


In [8]:
from newspaper import Article

with connection.cursor() as cursor:
    sql = """
    SELECT
        dateadded,
        avgtone,
        SOURCEURL,
        GLOBALEVENTID,
        EventCode,
        EventRootCode,
        Actor1Code,
        actor2code,
        GoldsteinScale,
        NumMentions 
    FROM
        tm_fintech_all_clean limit 10
    """
    cursor.execute(sql)
    #print(sql)
    result = cursor.fetchall()
    
    for r in result:
        try:
            article = Article(r['sourceurl'])
            article.download()
            article.parse()
            judul = article.title.encode('utf-8')
            isi_berita = article.text.encode('utf-8')
            print(r['sourceurl'])
            print(article.title)
            
#             print("--")
#             print(r['sourceurl'])
#             print(article.title)
#             print(article.text)
#             print("--")
#             with connection.cursor() as cursor:
#                 sql = "UPDATE gojek SET title=%s,isi_berita=%s WHERE GLOBALEVENTID=%s"
#                 cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
#                 connection.commit()
#                 print("Update", r["GLOBALEVENTID"])
            if (judul!='') :
                print("ada isi")
                print("Update", r["GLOBALEVENTID"])
            else:
                print("tidak ada")
#             with connection.cursor() as cursor:
#                 sql = "UPDATE gojek SET title=%s,isi_berita=%s WHERE GLOBALEVENTID=%s"
#                 cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
#                 connection.commit()
#                 print("Update", r["GLOBALEVENTID"])
        except:
            #row = row + 1
            pass

In [2]:
from newspaper import Article
article = Article('http://bali.coconuts.co/2015/11/30/go-jek-drivers-angrily-demonstrate-denpasar-getting-fined-accused-creating-fake-orders', language='zh')
article.download()
article.parse()
print(article.title)
print(article.text)

Go-Jek drivers angrily demonstrate in Denpasar for getting fined, accused of creating fake orders

